  <h1><center>Practicing Linear Regression on the US Housing Price Data "USA_Housing.csv"<br>
        Catawba College Data Mining Class</center><br>
        <h2> Research question:  Using public data on USA Housing, can we accurately predict housing price based on other features such as number of bedrooms, area population, area income?<br>
<h2>Group Members (add names for each member in the group)<br>
1.<br><br>
2.<br><br>
3.<br><br>
4.<br></h1></h2>



<p><h3>DIRECTIONS:<br><br>
    1.  Carefully read Chapters 15 and 16 in Data 8:<br>
    <a href="https://www.inferentialthinking.com/chapters/15/Prediction.html">Chapter 15</a><br>
        <a href="https://www.inferentialthinking.com/chapters/16/Inference_for_Regression.html">Chapter 16</a><br>
2.  Review and discuss (with your group) the following markdown (text narrative) and code<br>
3. Provide responses to the questions at the end of the notebook.<br>
    4.  Download your notebook and submit in Blackboard by the due date.<br></h3></h1></p>


## PART 1: Introduction to Linear Regression Modeling

In statistics, linear regression is a popular approach for modeling the relationship between a dependent variable y and one or more explanatory variables (or independent variables) denoted X. The case of one explanatory variable is called simple linear regression. For more than one explanatory variable, the model is labeled multiple linear regression.

In linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Such models are called linear models. 

Linear regression models are often fitted using the least squares approach, but they may also be fitted in other ways (see note at end). Additionally, the least squares approach can be used to fit models that are not linear models. Although the terms "least squares" and "linear model" are closely linked, they are not synonymous.

NOTE:  Other modelings for linear regression include minimizing the "lack of fit" in some other form (as with least absolute deviations regression), or by minimizing a penalized version of the least squares loss function as in ridge regression ($L_2$-norm penalty) and lasso ($L_1$-norm penalty). This is a topic for advanced discussion!

## PART 2: Coding a Linear Regression Model in Python with Jupyter using the USA_Housing dataset.

### 2. A. There are two main ways to perform linear regression in Python — with Statsmodels and scikit-learn. We will use scikit-learn.

### Import packages and dataset (read code comments and run each code block, some will have output)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


### 2. B.  Load the CSV file and examine the first few records

In [ ]:
# df will be the name of the file that we input
df = pd.read_csv("https://raw.githubusercontent.com/catawba-data-mining/CIS-3902-Data-Mining/main/USA_Housing.csv")
# df = pd.read_csv("USA_Housing.csv")
df.head()

### 2. C.  Data Exploration and Preprocessing.  Data exploration helps you learn more about the features of the dataset, data preprocessing is where you prepare the data for modeling.  We will be examining basic information on the data set - our ulitmate goal will be to predict Price for houses based on the other numeric variables such as average area income, average area house age, etc.

In [ ]:
# 1.
# we will use info to get basic information
# note the numeric and non-numeric attributes
df.info(verbose=True)

In [ ]:
# 2.
#next use df.describe() to get the statistical summary of the various features of the data set
#the percentiles show the data values up to a certain percentile
#for example 10% of the Average Area Number of Bedrooms is 0 up to 2.31
df.describe(percentiles=[0.1,0.25,0.5,0.75,0.9])

In [ ]:
# 3.
# even though we have seen them, lets use df.columns to see the column names
df.columns

**Pairplots using seaborn** Examine Price on the x axis and the relationship of the other individual variables on the y axis.  Do you see any linear trends? Do any other variables show a linear trend with each other?

In [ ]:
# 4.  We are using seaborn to get pairplots for all of the data that is numeric!

sns.pairplot(df)

**Distribution of price (the predicted quantity)** We want to first check the distribution of the variable we are trying to predict.  Why?   It is useful to understand the distribution of predictor variables to find influential outliers or concentrated values as this can affect the accuracy of the regression model. A highly skewed independent variable may be made more symmetric with a transformation which may include removing outliers or applying a transformation to allow the distribution to be closer to a normal distribution.

In [ ]:
# 4.  Let's look at the distribution of the variable we want to predict:  Price
# here is a histogram

df['Price'].plot.hist(bins=25,figsize=(8,4))

In [ ]:
# 5. another look -- there are also additional ways to do this
df['Price'].plot.density()

**Correlation matrix and heatmap** Price seems to be normally distributed.  Let's continue our exploration by looking at correlations between variables.  We will use df.corr() to look at correlation coefficients and heatmaps are always fun!

In [ ]:
# 6.
# curious about what this actually is calculating? 
# Pandas dataframe.corr () is used to find the pairwise correlation of all columns 
# in the dataframe. Any na values are automatically excluded. 
# This is for any non-numeric data type columns in the dataframe of course!
df.corr()

In [ ]:
# 7.
# Here is the heatmap using the amazing functionality of seaborn
# anything black is closer to 0, with light pink at 1 for correlation.  The diagonal
# column through the middle shows the variables correlated to themselves so
# of course this is at 1.
# we are looking for variables that are at .8 (orange) or above.
plt.figure(figsize=(10,7))
sns.heatmap(df.corr(),annot=True,linewidths=2)

### 3.  Preparing the Data for Modeling.  

We are going to put the independent variables in a data frame which we will name "X", and the dependent variable Price in y - we want to develop a predictive model using linear regression for y or Price.

Here are the steps:

3. A. through 3. E.:  Split the data into the two data frames - X and y.  Remove the non-numeric address variable.
3. F. through 3. H.:  Create two versions of this:  training and test data sets.  We use the training data set to train the model - this set includes the predictor variable Price and fits the model based on knowing the outcome. Then the test data set is used to "test" the model that is developed.  The test data set makes predictions on Price (without knowing Price) and then we compare the Predictions to the actual Price that we know is in the test data.  

Training data is usually around 70% of the data, then the testing data is 30% and the records do not overlap, they are separate.  So if you have 100 records, 70 will be in train, and 30 in test.


3. A. 

** Make a list of data frame column names** Python List items are indexed, the first item has index [0], the second item has index [1] etc. We will be using a list to split the data into two data frames ultimately:  X and y.  

Remember Price is the outcome variable, also called the response or dependent variable -- it is what we hope to predict with linear regression.  The other variables are the predictors, or explanatory or independent variables.

In regression analysis, the dependent variable is denoted "y" and the independent variables are denoted by "X".

We are going to first create a list of the column names (we will drop the address) and place all but Price in X for the independent variables.  y will hold Price, our response or dependent variable.

Want to learn more about data frames?  <a href="https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe.htm">Tutorial on Data Frames</a>

In [ ]:
# 3. A.
l_column = list(df.columns) # Making a list out of all of the column names
len_feature = len(l_column) # Length of column vector list
l_column

** Put all the numerical features in X (data frame), Price in y, ignore Address which is string and of course will not be used for our model for linear regression**

The following code is referencing the index values of the data frame and the first feature is at index 0.  X then gets features 0 to 5 so X includes everything but Price and Address from the list above.

Y gets feature 7 (len_feature) - 2 or 5 which is the feature in position 6 or price since the indexes are zero based

In [ ]:
# 3. B.
X = df[l_column[0:len_feature-2]]
y = df[l_column[len_feature-2]]

** Let's take a look **

In [ ]:
# 3. C.
print("Feature set size:",X.shape)
print("Variable set size:",y.shape)

In [ ]:
# 3. D.
X.head()

In [ ]:
# 3. E. 
y.head()

### Test-train split
#### Now we will split our data into a training and testing split - 70% training and 30% testing.  We will train the model on the training split then evaluate the model on the test split.

#### NOTE:  This is called a supervised model because we have a predictor variable "Price" that is supervising our learning - we know what the outcome is for our data (not for new data of course) and we are using this information to train our linear regression model in a "supervised" way!

#### The train and test sets are vital to ensure that your supervised learning model is able to generalize well to new data.

**There are two main ways to perform linear regression in Python — with Statsmodels and scikit-learn. We are using scikit-learn and will start by importing some of the functionality.**

**Import train_test_split function from scikit-learn**

In [ ]:
#3. F.
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

**Create X and y train and test splits in one command using a split ratio and a random seed**

In [ ]:
#3. G. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

**Check the size and shape of train/test splits (it should be in the ratio as per test_size parameter above)**

In [ ]:
#3. H. 
print("Training feature set size:",X_train.shape)
print("Test feature set size:",X_test.shape)
print("Training variable set size:",y_train.shape)
print("Test variable set size:",y_test.shape)

### 4. Model fit and training - we are ready to run our linear regression model!

**Import linear regression model estimator from scikit-learn and instantiate**

In [ ]:
# 4. A.
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
#4. B.
lm = LinearRegression() # Creating a Linear Regression object 'lm'

**Fit the model on to the instantiated object using our training data which is 70% of the data.  Note how we are using X train with the independent features and Y train which has the matching independent variable for each of the rows (Price)**

In [ ]:
#4. C.
lm.fit(X_train,y_train) 
# Fit the linear model on to the 'lm' object itself i.e. no need to set this to another variable

# **Check the results - think about what you have learned about linear regression from statistics class and this class.  We are going to get the intercept and coefficients from our modeling step and put them in a DataFrame**

Notice how lm has all of the good information for us!  

In [ ]:
#4. D. let's print some of the actual  predictions that were made
predictions = lm.predict(X)
print(predictions)

In [ ]:
#4. E.
print("The intercept term of the linear model:", lm.intercept_)

In [ ]:
#4. F.
# Let's print the coefficients
cdf = pd.DataFrame(data=lm.coef_, index=X_train.columns, columns=["Coefficients"])
cdf

### 5.  Model Evaluation - how did we do with the train data? Let's look at the statistics.  Need a quick review of standard errors, t-statistic and R squared, in other words evaluation measures for linear regression and other models?  Check here:  <a href="https://www.youtube.com/watch?v=7MAuojBTF-g">Khan Academy Calculating T-Statistic for Slope of Regression Line</a>.  Please note:  coefficient is sometimes used as a alternate tterm for slope.
### Calculation of standard errors and t-statistic for the coefficients
### Important:  see list of features in the order of importance for predicting the house price

In [ ]:
#5. A. Coefficients, Standard Error, t-statistic
n=X_train.shape[0]
k=X_train.shape[1]
dfN = n-k
train_pred=lm.predict(X_train)
train_error = np.square(train_pred - y_train)
sum_error=np.sum(train_error)
se=[0,0,0,0,0]
for i in range(k):
    r = (sum_error/dfN)
    r = r/np.sum(np.square(X_train[list(X_train.columns)[i]]-X_train[list(X_train.columns)[i]].mean()))
    se[i]=np.sqrt(r)
cdf['Standard Error']=se
cdf['t-statistic']=cdf['Coefficients']/cdf['Standard Error']
cdf

In [ ]:
# 5. B. let's print the features by the t-statistic, the higher the better the value is at predicting price
print("Features arranged in the order of importance for predicting the house price\n",'-'*90,sep='')
l=list(cdf.sort_values('t-statistic',ascending=False).index)
print(' > \n'.join(l))

In [ ]:
#5. C. Why not create some scatter plots for each feature with respect to price?
l=list(cdf.index)
from matplotlib import gridspec
fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(2,3)
#f, ax = plt.subplots(nrows=1,ncols=len(l), sharey=True)
ax0 = plt.subplot(gs[0])
ax0.scatter(df[l[0]],df['Price'])
ax0.set_title(l[0]+" vs. Price", fontdict={'fontsize':20})

ax1 = plt.subplot(gs[1])
ax1.scatter(df[l[1]],df['Price'])
ax1.set_title(l[1]+" vs. Price",fontdict={'fontsize':20})

ax2 = plt.subplot(gs[2])
ax2.scatter(df[l[2]],df['Price'])
ax2.set_title(l[2]+" vs. Price",fontdict={'fontsize':20})

ax3 = plt.subplot(gs[3])
ax3.scatter(df[l[3]],df['Price'])
ax3.set_title(l[3]+" vs. Price",fontdict={'fontsize':20})

ax4 = plt.subplot(gs[4])
ax4.scatter(df[l[4]],df['Price'])
ax4.set_title(l[4]+" vs. Price",fontdict={'fontsize':20})

**R-square of the model fit**

**R-square value is a formula that determines how much a variable’s behavior can explain the behavior of another variable.**

In [ ]:
#5. D.
print("R-squared value of this fit:",round(metrics.r2_score(y_train,train_pred),3))

### 6.  It is time to put our model developed from the train data to the test!  Remember that test data set that had 30% of our data?  Let's run the model on our test data to get predictions, then match the predictions to the actual values from the data to see how effective our model is on "new" data.  We will see prediction, error estimate, and regression evaluation matrices.  One thing to look out for . . . something called overfitting - it's when the model learns the test data "too well" and can't predict well. Remember this term!

**Prediction using the lm model and our test data set**

In [ ]:
#6. A.
predictions = lm.predict(X_test)
print ("Type of the predicted object:", type(predictions))
print ("Size of the predicted object:", predictions.shape)

**Scatter plot of predicted price and y_test set to see if the data fall on a 45 degree straight line - we are examining the model predictions on the test data and the actual values of price - very exciting!**

In [ ]:
#6. B.
plt.figure(figsize=(10,7))
plt.title("Actual vs. predicted house prices",fontsize=25)
plt.xlabel("Actual test set house prices",fontsize=18, color="red")
plt.ylabel("Predicted house prices", fontsize=18)
plt.scatter(x=y_test,y=predictions)

**More evaluation - the scatter plot looks pretty good!  Plotting histogram of the residuals i.e. predicted errors (expect a normally distributed pattern)**

In [ ]:
#6. C. 
plt.figure(figsize=(10,7))
plt.title("Histogram of residuals to check for normality",fontsize=25)
plt.xlabel("Residuals",fontsize=18)
plt.ylabel("Kernel density", fontsize=18)
sns.distplot([y_test-predictions])

**Scatter plot of residuals and predicted values (Homoscedasticity - The assumption of homoscedasticity (meaning “same variance”) is central to linear regression models.)**<p>Why do we care?</p><p><a href="https://www.statisticshowto.datasciencecentral.com/heteroscedasticity-simple-definition-examples/">Heteroscedasticity and Homoscedasticity</a></p>
### This is important!  Take the time to review the article!

In [ ]:
#6. D. 
plt.figure(figsize=(10,7))
plt.title("Residuals vs. predicted values plot (Homoscedasticity)\n",fontsize=25)
plt.xlabel("Predicted house prices",fontsize=18)
plt.ylabel("Residuals", fontsize=18)
plt.scatter(x=predictions,y=y_test-predictions)

**Regression evaluation metrices**

In [ ]:
#6. E.
print("Mean absolute error (MAE):", metrics.mean_absolute_error(y_test,predictions))
print("Mean square error (MSE):", metrics.mean_squared_error(y_test,predictions))
print("Root mean square error (RMSE):", np.sqrt(metrics.mean_squared_error(y_test,predictions)))

In [ ]:
#6. F. 
print("R-squared value of predictions:",round(metrics.r2_score(y_test,predictions),3))

**R-square value** - the R-sqared value (#6. F) of predictions is .919
<p>Evaluating R-squared - the coefficient of Determination</p><p><a href="https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/coefficient-of-determination-r-squared/">R-squared</a></p>

QUESTIONS - ANSWER THE FOLLOWING QUESTIONS BY INSERTING MARKDOWN CELLS BELOW WITH THE QUESTION NUMBER AND THE ANSWER

1.  What type of variables are acceptable for linear regression for the X variable(s)?  The Y variable?<br>
2.  What variable was discarded from the original dataset and why?
3.  What is the purpose of the training and test data set?
4.  What is overfitting and what can cause it?
5.  What is the purpose of plotting the variables to check for homoscedasticity?
6.  This model produced a coefficient of determination of .919.  Is this good?  What does it mean?

1.

2.

3.

4.

5.

6.